<img align="right" src="images/dans-small.png"/>
<img align="right" src="images/tf-small.png"/>
<img align="right" src="images/etcbc.png"/>

# BHSA and OpenScriptures bridge

Both the BHSA and the OpenScriptures represent efforts to add linguistic markup to the Hebrew Bible.

The BHSA is the product of years of encoding work by researchers, in a strongly algorithmic fashion, although
not without human decisions at the micro level.

OpenScriptures represents a crowd sourced approach.

Regardless of theoretical considerations on the validity of these approaches, it is worthwhile to be able to compare them.
Moreover, for some research problems, it might be helpful to use both encodings in one toolkit.

In this repo we develop a way of doing exactly this.

We make a link between the morphology in the
[Openscriptures](http://openscriptures.org)
and the linguistics in the [BHSA](https://github.com/ETCBC/bhsa).

We proceed as follows:

* extract the morphology from the files in
  [openscriptures/morphhb/wlc](https://github.com/openscriptures/morphhb/tree/master/wlc)
* link the words in the openscripture files to slots in the BHSA
* compile the openscripture morphology data into a TF feature file.

With this in hand, we have the OpenScriptures morphology in Text-Fabric, aligned to the BHSA.
That opens the way for further comparisons, which take the actual morphology into account.

When we first made the comparison, in 2017, only 88% of the OpenScriptures Morphology was fixed.

In 2021 we have pulled the same repository again, and used a new version of the BHSA as well.
It turns out the 100% of the words have been morphologically annotated by OpenScriptures now.

## Application

This notebook sets the stage for focused comparisons between the BHSA features on words and the OSM morphology.

See

* [category](category.ipynb)
* [language](language.ipynb)
* [part-of-speech](part-of-speech.ipynb)
* [verb](verb.ipynb)

In [2]:
import os
import collections
from glob import glob
from lxml import etree
from itertools import zip_longest
from functools import reduce
from unicodedata import normalize, category

from tf.fabric import Fabric
from tf.core.helpers import rangesFromSet


# Loading BHSA

In [3]:
BHSA = os.path.expanduser("~/github/etcbc/bhsa")
BHSADir = "{}/tf".format(BHSA)
# VERSION = "2017"
VERSION = "2021"

REPO = os.path.expanduser("~/github/etcbc/bridging")
TFDIR = "{}/tf/{}".format(REPO, VERSION)

TF = Fabric(locations="{}/tf/{}".format(BHSA, VERSION), modules=[""])
api = TF.load(
    """
    book
    g_cons_utf8 g_word_utf8
"""
)
api.makeAvailableIn(globals())


This is Text-Fabric 9.0.3
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

115 features found and 0 ignored
  0.00s loading features ...
   |     0.00s Dataset without structure sections in otext:no structure functions in the T-API
  3.51s All features loaded/computed - for details use TF.isLoaded()


[('Computed',
  'computed-data',
  ('C Computed', 'Call AllComputeds', 'Cs ComputedString')),
 ('Features', 'edge-features', ('E Edge', 'Eall AllEdges', 'Es EdgeString')),
 ('Fabric', 'loading', ('TF',)),
 ('Locality', 'locality', ('L Locality',)),
 ('Nodes', 'navigating-nodes', ('N Nodes',)),
 ('Features',
  'node-features',
  ('F Feature', 'Fall AllFeatures', 'Fs FeatureString')),
 ('Search', 'search', ('S Search',)),
 ('Text', 'text', ('T Text',))]

# Reading Open Scriptures Morphology

In [4]:
NB_DIR = os.getcwd()
OS_BASE = os.path.expanduser("~/github/openscriptures/morphhb/wlc")
os.chdir(OS_BASE)

## Mapping the book names
OSM uses abbreviated book names.
We map them onto the (latin) book names of the BHSA.

Here is a list of the BHSA books.

In [5]:
bhsBooks = [F.book.v(n) for n in F.otype.s("book")]
print(" ".join(bhsBooks))

Genesis Exodus Leviticus Numeri Deuteronomium Josua Judices Samuel_I Samuel_II Reges_I Reges_II Jesaia Jeremia Ezechiel Hosea Joel Amos Obadia Jona Micha Nahum Habakuk Zephania Haggai Sacharia Maleachi Psalmi Iob Proverbia Ruth Canticum Ecclesiastes Threni Esther Daniel Esra Nehemia Chronica_I Chronica_II


The next cell can be used to retrieve the OSM book names,
from which the ordered list `osmBooks` below can be composed manually.

In [6]:
osmBookSet = set(fn[0:-4] for fn in glob("*.xml") if fn != "VerseMap.xml")
print(" ".join(sorted(osmBookSet)))

1Chr 1Kgs 1Sam 2Chr 2Kgs 2Sam Amos Dan Deut Eccl Esth Exod Ezek Ezra Gen Hab Hag Hos Isa Jer Job Joel Jonah Josh Judg Lam Lev Mal Mic Nah Neh Num Obad Prov Ps Ruth Song Zech Zeph


We list the books in the "canonical" order (as given in the BHSA).

In [7]:
osmBooks = """
Gen Exod Lev Num Deut
Josh Judg 1Sam 2Sam 1Kgs 2Kgs
Isa Jer Ezek Hos Joel Amos Obad
Jonah Mic Nah Hab Zeph Hag Zech Mal
Ps Job Prov Ruth Song Eccl Lam Esth
Dan Ezra Neh 1Chr 2Chr
""".strip().split()

Now we can construct the mapping, both ways.

In [8]:
osmBookFromBhs = {}
bhsBookFromOsm = {}
for (i, bhsBook) in enumerate(bhsBooks):
    osmBook = osmBooks[i]
    osmBookFromBhs[bhsBook] = osmBook
    bhsBookFromOsm[osmBook] = bhsBook

## Consonantal matters

For alignment purposes, we reduces all textual material to their consonantal representation.
Sometimes we need to blur the distinction between final consonants and their normal counterparts.

In order to strip consonants from all their diacritical marks, we use unicode denormalization and
unicode character categories.

In [9]:
NS = "{http://www.bibletechnologies.net/2003/OSIS/namespace}"
NFD = "NFD"
LO = "Lo"

finals = {
    "ך": "כ",
    "ם": "מ",
    "ן": "נ",
    "ף": "פ",
    "ץ": "צ",
}

finalsI = {v: k for (k, v) in finals.items()}

`toCons(s)`: strip all pointing (accents, vowels, dagesh, s(h)in dot) from all characters in string `s`.

`final(c)`: replace consonant `c` by its final counterpart, if there is one, otherwise return `c`.

`finalCons(s)`: replace the last character of `s` by its final counterpart.

`unFinal(s)`: replace all consonants in `s` by their non-final counterparts.

In [10]:
def toCons(s):
    return "".join(c for c in normalize(NFD, s) if category(c) == LO)

In [11]:
def final(c):
    return finalsI.get(c, c)

In [12]:
def finalCons(s):
    return s[0:-1] + final(s[-1])

In [13]:
def unFinal(s):
    return "".join(finals.get(c, c) for c in s)

## Read OSM books

We are going to read the OSM files.
They correspond to books.

We drill down to verse nodes and pick up the `<w>` elements.
What we need from these elements is the full text content and the attributes
`lemma` and `morph`.

We ignore markup within the full text content of the `<w>` elements.

The material we extract, may contain `/`.
We split the text content and the `lemma` and `morph` content on `/`, and recombine the resulting parts in
OSM morpheme entries, having each a full-text bit, a morph bit and a lemma bit.

Caveat: when splitting the morpheme string, we should first split off the first character, which indicates language,
and then add it to all the parts!

So, one `<w>` element may give rise to several morpheme entries.

The full text is fully pointed. We also compute a consonantal version of the full text and store it
within the morpheme entries.

We end up with a list, `osmMorphemes` of morpheme entries.

In passing, we count the `<w>` elements without `morph` attributes, and those without textual content.

We also store the book, chapter, verse and sequence number of the `<w>` element in each entry.

In [14]:
def readOsmBook(osmBook, osmMorphemes, stats):
    infile = "{}.xml".format(osmBook)
    parser = etree.XMLParser(remove_blank_text=True, ns_clean=True)
    root = etree.parse(infile, parser).getroot()
    osisTextNode = root[0]
    divNode = osisTextNode[1]
    chapterNodes = list(divNode)
    print(
        "reading {:<5} ({:<15}) {:>3} chapters".format(
            osmBook, bhsBookFromOsm[osmBook], len(chapterNodes)
        )
    )
    ch = 0
    for chapterNode in chapterNodes:
        if chapterNode.tag != NS + "chapter":
            continue
        ch += 1
        vs = 0
        for verseNode in list(chapterNode):
            if verseNode.tag != NS + "verse":
                continue
            vs += 1
            w = 0
            for wordNode in list(verseNode):
                if wordNode.tag != NS + "w":
                    continue
                w += 1
                lemma = wordNode.get("lemma", None)
                morph = wordNode.get("morph", None)
                text = "".join(x for x in wordNode.itertext())

                lemmas = lemma.split("/") if lemma is not None else []
                morphs = morph.split("/") if morph is not None else []
                if len(morphs) > 1:
                    lang = morphs[0][0]
                    morphs = [morphs[0]] + [lang + m for m in morphs[1:]]
                texts = text.split("/") if text is not None else []
                # zip_longest accomodates for unequal lengths of its operands
                # for missing values we fill in ''
                for (lm, mph, tx) in zip_longest(lemmas, morphs, texts, fillvalue=""):
                    txc = None if tx is None else toCons(tx)
                    osmMorphemes.append((tx, txc, mph, lm, osmBook, ch, vs, w))
                    if not mph:
                        stats["noMorph"] += 1
                    if not tx:
                        stats["noContent"] += 1

That was the definition of the read function, now we are going to execute it.

In [15]:
osmMorphemes = []
stats = dict(noMorph=0, noContent=0)

for bn in F.otype.s("book"):
    bhsBook = T.sectionFromNode(bn, lang="la")[0]
    osmBook = osmBookFromBhs[bhsBook]
    readOsmBook(osmBook, osmMorphemes, stats)

print(
    """
BHS words:       {:>6}
Collected words: {:>6}
No morphology:   {:>6}
No content:      {:>6}
{} % of the words are morphologically annotated.
""".format(
        F.otype.maxSlot,
        len(osmMorphemes),
        stats["noMorph"],
        stats["noContent"],
        round(
            100
            * (len(osmMorphemes) - stats["noMorph"] - stats["noContent"])
            / len(osmMorphemes)
        ),
    )
)

reading Gen   (Genesis        )  50 chapters
reading Exod  (Exodus         )  40 chapters
reading Lev   (Leviticus      )  27 chapters
reading Num   (Numeri         )  36 chapters
reading Deut  (Deuteronomium  )  34 chapters
reading Josh  (Josua          )  24 chapters
reading Judg  (Judices        )  21 chapters
reading 1Sam  (Samuel_I       )  31 chapters
reading 2Sam  (Samuel_II      )  24 chapters
reading 1Kgs  (Reges_I        )  22 chapters
reading 2Kgs  (Reges_II       )  25 chapters
reading Isa   (Jesaia         )  66 chapters
reading Jer   (Jeremia        )  52 chapters
reading Ezek  (Ezechiel       )  48 chapters
reading Hos   (Hosea          )  14 chapters
reading Joel  (Joel           )   4 chapters
reading Amos  (Amos           )   9 chapters
reading Obad  (Obadia         )   1 chapters
reading Jonah (Jona           )   4 chapters
reading Mic   (Micha          )   7 chapters
reading Nah   (Nahum          )   3 chapters
reading Hab   (Habakuk        )   3 chapters
reading Ze

To give an impression of the contents of this list, we show the first few members.
The column specification is:

    consonantal fully-pointed morph lemma book chapter verse w-number

In [16]:
list(osmMorphemes[0:15])

[('בְּ', 'ב', 'HR', 'b', 'Gen', 1, 1, 1),
 ('רֵאשִׁ֖ית', 'ראשית', 'HNcfsa', '7225', 'Gen', 1, 1, 1),
 ('בָּרָ֣א', 'ברא', 'HVqp3ms', '1254 a', 'Gen', 1, 1, 2),
 ('אֱלֹהִ֑ים', 'אלהים', 'HNcmpa', '430', 'Gen', 1, 1, 3),
 ('אֵ֥ת', 'את', 'HTo', '853', 'Gen', 1, 1, 4),
 ('הַ', 'ה', 'HTd', 'd', 'Gen', 1, 1, 5),
 ('שָּׁמַ֖יִם', 'שמים', 'HNcmpa', '8064', 'Gen', 1, 1, 5),
 ('וְ', 'ו', 'HC', 'c', 'Gen', 1, 1, 6),
 ('אֵ֥ת', 'את', 'HTo', '853', 'Gen', 1, 1, 6),
 ('הָ', 'ה', 'HTd', 'd', 'Gen', 1, 1, 7),
 ('אָֽרֶץ', 'ארץ', 'HNcbsa', '776', 'Gen', 1, 1, 7),
 ('וְ', 'ו', 'HC', 'c', 'Gen', 1, 2, 1),
 ('הָ', 'ה', 'HTd', 'd', 'Gen', 1, 2, 1),
 ('אָ֗רֶץ', 'ארץ', 'HNcbsa', '776', 'Gen', 1, 2, 1),
 ('הָיְתָ֥ה', 'היתה', 'HVqp3fs', '1961', 'Gen', 1, 2, 2)]

# Alignment

We now have to face the task to map the BHSA words to the OSM morphemes.

We will encounter the challenge that at some spots the consonantal contents of the WLC (the source of the OSM)
is different from that of the BHS, the source of the BHSA.

Another challenge is that at some points the analysis behind the OSM differs from that of the BHSA in such a way
that the BHSA has a word-split within an OSM morpheme.

Yet another source of problems is that the BHSA inserts "empty" articles in places where the pointing in the
surrounding material allows to conclude that an article is present, although it does not have a consonantal presence anymore.

We need a function to quickly show what is going on in difficult spots.

`showCase(w, j, ln)` shows the BHSA from word `w` onwards, and the OSM from morpheme `j` onwards.
It lists `ln` positions in both sources.

In [17]:
def showCase(w, j, ln):
    print(T.sectionFromNode(w))
    print("BHS")
    for n in range(w, w + ln):
        print("word  {} = [{}]".format(n, toCons(F.g_cons_utf8.v(n))))
    print("OSM")
    for n in range(j, j + ln):
        print("morph {} = [{}]".format(n, osmMorphemes[n][1]))

We also define another function to easy inspect difficult spots.

`BHSvsOSM(ws, js)` compares the BHSA words specified by list `ws` with the OSM morphemes
specified by list `js`.

In [18]:
def BHSvsOSM(ws, js):
    print(
        "{}\n{:<25}BHS {:<30} = {}\n{:<25}OSM {:<30} = {}".format(
            "{} {}:{}".format(*T.sectionFromNode(ws[0])),
            " ",
            ", ".join(str(w) for w in ws),
            "/".join(F.g_word_utf8.v(w) for w in ws),
            " ",
            ", ".join("w{}".format(osmMorphemes[j][7]) for j in js),
            "/".join(osmMorphemes[j][0] for j in js),
        )
    )

Before we flesh out the alignment algorithm,
let's find the first point where BHSA and OSM diverge.

In [19]:
for (i, w) in enumerate(F.otype.s("word")):
    bhs = toCons(F.g_cons_utf8.v(w))
    osm = osmMorphemes[i][1]
    if bhs != osm:
        print("Mismatch at BHS-{} OSM-{}: bhs=[{}] osm=[{}]".format(w, i, bhs, osm))
        break

Mismatch at BHS-62 OSM-61: bhs=[] osm=[אור]


In [20]:
showCase(61, 60, 5)

('Genesis', 1, 5)
BHS
word  61 = [ל]
word  62 = []
word  63 = [אור]
word  64 = [יום]
word  65 = [ו]
OSM
morph 60 = [ל]
morph 61 = [אור]
morph 62 = [יום]
morph 63 = [ו]
morph 64 = [ל]


This is a case of an empty article in the BHSA.
Let's circumvent this, and move on.

In [21]:
j = -1
for w in F.otype.s("word"):
    bhs = toCons(F.g_cons_utf8.v(w))
    if bhs == "":
        continue
    j += 1
    osm = osmMorphemes[j][1]
    if bhs != osm:
        print(
            """Mismatch at BHS-{} OSM-{}:\nbhs=[{}]\nos=[{}]""".format(w, j, bhs, osm)
        )
        break

Mismatch at BHS-194 OSM-187:
bhs=[מינו]
os=[מינ]


In [22]:
showCase(194, 187, 5)

('Genesis', 1, 11)
BHS
word  194 = [מינו]
word  195 = [אשר]
word  196 = [זרעו]
word  197 = [בו]
word  198 = [על]
OSM
morph 187 = [מינ]
morph 188 = [ו]
morph 189 = [אשר]
morph 190 = [זרע]
morph 191 = [ו]


Here we bump into the fact that the BHSA deals with whole words, and the OSM splits into morphemes.
In this case, the pronominal suffix is treated as a separate morpheme.

## Algorithm

We have to develop a way of aligning each BHS word with one or more OSM morphemes.

For each BHS word, we grab OSM morphemes until all consonants in the BHS word have been matched.
If needed, we grab additional BHS words
when the current OSM string happens to be longer than the current BHS word.

We will encounter cases where this method breaks down: exceptions.
We will collect them for later inspection.

The exceptions are coded as follows:

If `w: n` is in the dictionary of exceptions, it means that slot (word) `w` in the BHSA is different from its counterpart morpheme(s) in the OSM.

If `n > 0`, that many OSM morphemes will be gobbled to align with slot `w`.

If `n < 0`, that many slots from `w` will be gobbled to match the current OSM morpheme.

There are various subtleties involved, see the inline content in the code below.

In [42]:
allExceptions = {
    "2017": {
        215253: 1,
        266189: 1,
        287360: 2,
        376865: 1,
        383405: 2,
        384049: 1,
        384050: 1,
        405102: -2,
    },
    "2021": {
        215256: 1,
        266192: 1,
        287363: 2,
        376869: 1,
        383409: 2,
        384053: 1,
        384054: 1,
        405108: -2,
    },
}

exceptions = allExceptions[VERSION]

In [43]:
# index in the osmMorphemes list
j = -1

# mapping from BHSA slot numbers to OSM morphemes indices
osmFromBhs = {}

u = None
remainingErrors = False
for w in F.otype.s("word"):
    # the previous iteration may have already dealt with this word
    # in that case, we skip to the next word
    # the signal is: w <= u
    if u is not None and w <= u:
        continue

    # we get the consonantal BHSA word string
    bhs = toCons(F.g_cons_utf8.v(w))

    # if the BHSA word is empty, we do not link it to any OSM morpheme
    # and continue
    if bhs == "":
        continue

    # we are going to collect OSM morphemes
    # as long as the consonantal reps of the morpheme fit into the BHSA word
    j += 1
    startJ = j
    startW = w
    osm = osmMorphemes[j][1]

    # but if the word is listed as exception, we collect as many morphemes
    # as specified in the exception
    maxGobble = exceptions.get(w, None)
    gobble = 1
    while (len(osm) < len(bhs) and bhs.startswith(osm)) or (
        maxGobble is not None and maxGobble > 0
    ):
        if maxGobble is not None and gobble >= maxGobble:
            break
        j += 1
        osm += osmMorphemes[j][1]
        gobble += 1

    # if the OSM morphemes have become longer than the BHSA word,
    # we eat up the following BHSA word(s)
    # we let u hold the new BHSA word position
    u = w
    gobble = 1
    while (len(osm) > len(bhs) and osm.startswith(bhs)) or (
        maxGobble is not None and maxGobble < 0
    ):
        if maxGobble is not None and gobble >= -maxGobble:
            break
        u += 1
        bhs += toCons(F.g_cons_utf8.v(u))
        gobble += 1
    gobble = 1

    # if the BHSA words exceed the OSM morphemes found so far, we draw in additional OSM morphemes
    # (for the last time)
    while len(osm) < len(bhs) and bhs.startswith(osm):
        if maxGobble is not None and gobble >= maxGobble:
            break
        j += 1
        osm += osmMorphemes[j][1]
        gobble += 1

    # now we have gathered a BHSA string of material, and an OSM string of material
    # We test if both strings are equal (modulo final consonant issues)
    # If not: alignment breaks down, we stop the loop and show the offending case.
    # The programmer should inspect the case and add an exception.
    if maxGobble is None and finalCons(bhs) != finalCons(osm):
        print(
            """Mismatch in {} at BHS-{} OS-{}->{}:\nbhs=[{}]\nos=[{}]""".format(
                "{} {}:{}".format(*T.sectionFromNode(w)),
                w,
                startJ,
                j,
                bhs,
                osm,
            )
        )
        showCase(w - 5, startJ - 5, j - startJ + 10)
        remainingErrors = True
        break

    # but if all is well, we link the BHSA words in question to the OSM morphemes in question
    # If the BHSA string contains multiple words, we link all those words to all morphemes
    for k in range(startW, u + 1):
        for m in range(startJ, j + 1):
            osmFromBhs.setdefault(k, []).append(m)

if not remainingErrors:
    print("Succeeded in aligning BHS with OSM")
    print(
        "{} BHS words matched against {} OSM morphemes with {} known exceptions".format(
            len(osmFromBhs),
            len(osmMorphemes),
            len(exceptions),
        )
    )

Succeeded in aligning BHS with OSM
420109 BHS words matched against 469440 OSM morphemes with 8 known exceptions


We have constructed in passing the mapping `osmFromBhs`,
which maps BHSA words onto corresponding sequences of OSM morphemes.
We also compute the inverse of this, `bhsFromOsm`.

In [44]:
# mapping from OSM morphemes (by index in osmMorphemes list) to BHSA slot numbers
# It is the inverse of osmFromBhs
bhsFromOsm = {}

for (w, js) in osmFromBhs.items():
    for j in js:
        bhsFromOsm.setdefault(j, []).append(w)
print("{} morphemes mapped in bhsFromOsm".format(len(bhsFromOsm)))

469440 morphemes mapped in bhsFromOsm


# Inspection of problems

We have encountered irregularities, but we want to make sure we have seen all potential
alignment problems.
We do this by adding a sanity check: find all cases where
the consonantal material in a BHSA word is not the
concatenation of the consonantal material in in its OSM morphemes.

We have now several irregularities to inspect.

1. **Multiplicity**
   * By looking into `bhsFromOsm` we can find the OSM morphemes that contain consonantal material
     from multiple BHSA words.
     These are interesting points of difference between the BHSA and OSM encoding, because
     in these cases the OSM produces other word/morpheme boundaries than the BHSA.
   * We also inspect cases where a BHSA word corresponds to more than two OSM morphemes.
1. **Consonantal Sanity**
   By analysis after the fact, we gather all consonantal discrepancies
1. **Exceptions**
   While developing the algorithm, we needed to invoke a small number of manual exceptions.


Now we want to make a comprehensive list of all problematic cases encountered during
alignment.

We will add the BHSA word numbers involved in a problematic case to the set `problematic`.
When we proceed to compare morphology, we will exclude the problematic cases.

In [45]:
problematic = set()

## Multiplicity
We gather the cases of multiple BHSA words against a single OSM morpheme.

In [46]:
multipleOSM = {}  # OSM morphemes in correspondence with multiple BHS slots
noOSM = {}  # OSM morphemes that do not correspond to any BHSA word

countMultipleOSM = (
    collections.Counter()
)  # how many times n BHSA words are linked to the same OSM morpheme

for (j, ws) in bhsFromOsm.items():
    nws = len(ws)
    if nws > 1:
        multipleOSM[j] = nws
        countMultipleOSM[nws] += 1
    elif nws == 0:
        noOSM.add(j)

print(
    "OSM morphemes without corresponding BHSA word:                {:>5}".format(
        len(noOSM)
    )
)
print(
    "OSM morphemes corresponding to multiple BHSA words:           {:>5}".format(
        len(multipleOSM)
    )
)
for (nws, amount) in sorted(countMultipleOSM.items()):
    print(
        "OSM morphemes corresponding to {} BHSA words:                  {:>5}".format(
            nws, amount
        )
    )

OSM morphemes without corresponding BHSA word:                    0
OSM morphemes corresponding to multiple BHSA words:             122
OSM morphemes corresponding to 2 BHSA words:                    115
OSM morphemes corresponding to 3 BHSA words:                      7


In [47]:
for j in multipleOSM:
    ws = bhsFromOsm[j]
    problematic |= set(ws)
    BHSvsOSM(ws, [j])

Genesis 24:65
                         BHS 12370, 12371                   = הַ/לָּזֶה֙
                         OSM w6                             = הַלָּזֶה֙
Genesis 37:19
                         BHS 20517, 20518                   = הַ/לָּזֶ֖ה
                         OSM w8                             = הַלָּזֶ֖ה
Genesis 50:10
                         BHS 28426, 28427                   = הָ/אָטָ֗ד
                         OSM w4                             = הָאָטָ֗ד
Genesis 50:11
                         BHS 28460, 28461                   = הָֽ/אָטָ֔ד
                         OSM w8                             = הָֽאָטָ֔ד
Numbers 13:21
                         BHS 78125, 78126                   = לְ/בֹ֥א
                         OSM w9                             = לְבֹ֥א
Numbers 34:8
                         BHS 91530, 91531                   = לְ/בֹ֣א
                         OSM w4                             = לְבֹ֣א
Deuteronomy 33:2
                         BHS 112265, 112266 

## Consonantal sanity
Which non-empty BHSA words are not the concatenation of their OSM morphemes?

We do not consider the cases where more than one BHSA word corresponds to an OSM morpheme,
because we have already gathered those cases above.

In [48]:
insaneBHS = set()  # alignment problems by BHSA slot number
insaneOSM = set()  # alignment problems by OSM morpheme index in osmMorphemes

# We compute the slot numbers of that are part of a multiple slot alignment to a morpheme
multipleBHS = reduce(set.union, (bhsFromOsm[j] for j in multipleOSM), set())

# Gather the insanities
for (w, js) in osmFromBhs.items():
    if w in multipleBHS:
        continue
    cw = toCons(F.g_cons_utf8.v(w))
    cjs = "".join(osmMorphemes[j][1] for j in js)
    if unFinal(cw) != unFinal(cjs):
        insaneBHS.add(w)
        insaneOSM |= set(js)
print("insane BHS words:     {:>4}".format(len(insaneBHS)))
print("insane OSM morphemes: {:>4}".format(len(insaneOSM)))

insane BHS words:        6
insane OSM morphemes:    8


In [49]:
for w in sorted(insaneBHS):
    problematic.add(w)
    js = osmFromBhs[w]
    BHSvsOSM([w], js)

Ezekiel 4:6
                         BHS 266192                         = ימוני
                         OSM w7                             = ימיני
Ezekiel 43:11
                         BHS 287363                         = צורתו
                         OSM w17, w17                       = צורת/י
Daniel 10:19
                         BHS 376869                         = כְ
                         OSM w10                            = בְ
Ezra 10:44
                         BHS 383409                         = נשׂאו
                         OSM w3, w3                         = נשא/י
Nehemiah 2:13
                         BHS 384053                         = הם
                         OSM w17                            = ה
Nehemiah 2:13
                         BHS 384054                         = פרוצים
                         OSM w17                            = מפרוצים


## More than two morphemes per word

Let's study the mapping of BHSA words to OSM morphemes in a bit more detail.
We are interested in the question: to how many morphemes can words map?

Later we shall see that we can deal with 1 and 2 morphemes per word.

We deem words that map to more than two morphemes problematic.

This turns out to be a very small minority.

In [50]:
morphemesPerWord = collections.Counter()
tooMany = set()
for (w, js) in osmFromBhs.items():
    n = len(js)
    morphemesPerWord[n] += 1
    if n > 2:
        tooMany.add(w)

for (ln, amount) in sorted(morphemesPerWord.items()):
    print("{:>2} morphemes per word: {:>6}".format(ln, amount))

 1 morphemes per word: 370680
 2 morphemes per word:  49400
 3 morphemes per word:     27
 4 morphemes per word:      2


In [51]:
for w in sorted(tooMany):
    js = osmFromBhs[w]
    BHSvsOSM([w], js)
    problematic.add(w)

Numbers 33:46
                         BHS 91209                          = עַלְמֹ֥ן דִּבְלָתָֽיְמָה
                         OSM w5, w6, w6                     = עַלְמֹ֥ן/דִּבְלָתָֽיְמָ/ה
Numbers 33:47
                         BHS 91213                          = עַלְמֹ֣ן דִּבְלָתָ֑יְמָה
                         OSM w2, w3, w3                     = עַלְמֹ֣ן/דִּבְלָתָ֑יְמָ/ה
Deuteronomy 10:6
                         BHS 99249                          = בְּאֵרֹ֥ת בְּנֵי־יַעֲקָ֖ן
                         OSM w4, w5, w6                     = בְּאֵרֹ֥ת/בְּנֵי/יַעֲקָ֖ן
Joshua 13:17
                         BHS 120713                         = בֵ֖ית בַּ֥עַל מְעֹֽון
                         OSM w9, w10, w11                   = בֵ֖ית/בַּ֥עַל/מְעֽוֹן
Joshua 15:32
                         BHS 121905                         = עַ֣יִן וְרִמֹּ֑ון
                         OSM w3, w4, w4                     = עַ֣יִן/וְ/רִמּ֑וֹן
Joshua 15:62
                         BHS 122150                         =

## Exceptions

Finally, we inspect the cases that correspond to the manual exceptions.

In [52]:
for (w, n) in exceptions.items():
    if n > 0:
        js = osmFromBhs[w]
        BHSvsOSM([w], js)
        problematic.add(w)
    else:
        j = osmFromBhs[w][0]
        ws = bhsFromOsm[j]
        BHSvsOSM(ws, [j])
        problematic |= set(ws)

Isaiah 9:6
                         BHS 215256                         = מרבה
                         OSM w1                             = םרבה
Ezekiel 4:6
                         BHS 266192                         = ימוני
                         OSM w7                             = ימיני
Ezekiel 43:11
                         BHS 287363                         = צורתו
                         OSM w17, w17                       = צורת/י
Daniel 10:19
                         BHS 376869                         = כְ
                         OSM w10                            = בְ
Ezra 10:44
                         BHS 383409                         = נשׂאו
                         OSM w3, w3                         = נשא/י
Nehemiah 2:13
                         BHS 384053                         = הם
                         OSM w17                            = ה
Nehemiah 2:13
                         BHS 384054                         = פרוצים
                         OSM w17        

Here is the number of problematic words in the BHSA that we will exclude from comparisons.

In [53]:
len(problematic)

259

# Missing morphology

The Open Scriptures data is not completely tagged with morphology.
We make a list of word nodes for which no morpheme has been tagged with morphology.

In [54]:
noMorphWords = set()
for w in F.otype.s("word"):
    hasMorph = False
    for j in osmFromBhs.get(w, []):
        if osmMorphemes[j][2]:
            hasMorph = True
            break
    if not hasMorph:
        noMorphWords.add(w)

In [55]:
len(noMorphWords)

6481

Let's get a feeling for how the non-tagged morphemes are distributed.
First we represent them as a list of intervals, using a utility function of TF,
and then we get an overview of the lengths of the intervals.

In [56]:
noMorphIntervals = rangesFromSet(noMorphWords)

noMorphLengths = collections.Counter()

for interval in noMorphIntervals:
    noMorphLengths[interval[1] - interval[0] + 1] += 1

print("Non-marked-up stretches having length x: y times")
for (ln, amount) in sorted(noMorphLengths.items()):
    print("{:>4}: {:>5}".format(ln, amount))

Non-marked-up stretches having length x: y times
   1:  6481


# Data generation

We now proceed to compile the OSM morphology into Text-Fabric features.

The basic idea is: create a feature `osm` and for each BHSA word, let it contain the contents of the
corresponding `morph` attribute in the OSM source.

There are several things to deal with, or not to deal with.

## Problematic cases
We will ignore the problematic cases. More precisely, whenever a BHSA word belongs to a problematic case,
as diagnosed before, we give fill its `osm` feature with the value `*`.

## Multiplicity of morphemes
There are BHSA words that do not correspond to OSM morphemes. The empty words. We will not give them an `osm` value.

There are BHSA words that correspond to more than two morphemes. We have added them to our problematic list.

The vast majority of BHSA words correspond to a single OSM morpheme.
The `osm` feature of those words will be filled
with the `morph` attribute part of the corresponding OSM morpheme. No problem here.

The remaining cases consist of BHSA words that correspond to exactly two morphemes.
We will use the value of the `morph` of the first morpheme to fill the `osm` feature for such words,
and we will make a new feature, `osm_sf` and fill it with the `morph` of the second morpheme.

So, we will create a TF module consisting of two features: `osm` and `osm_sf` (osm suffix).

Let's assemble the feature data.

In [57]:
osmData = {}
osm_sfData = {}
for (w, js) in osmFromBhs.items():
    if w in problematic:
        osmData[w] = "*"
        continue
    osmData[w] = osmMorphemes[js[0]][2]
    if len(js) > 1:
        osm_sfData[w] = osmMorphemes[js[1]][2]

And combine it with a bit of metadata.

In [58]:
nodeFeatures = dict(osm=osmData, osm_sf=osm_sfData)
metaData = dict(
    osm=dict(
        valueType="str",
        description="primary morphology string according to OpenScriptures",
        source="Open Scriptures",
        source_url="https://github.com/openscriptures/morphhb",
        conversion="notebook openscriptures in BHSA repo",
        conversion_author="Dirk Roorda",
        coreData="BHSA",
        coreVersion=VERSION,
    ),
    osm_sf=dict(
        valueType="str",
        description="secundary morphology string according to OpenScriptures",
        source="Open Scriptures",
        source_url="https://github.com/openscriptures/morphhb",
        conversion="notebook openscriptures in BHSA repo",
        conversion_author="Dirk Roorda",
        coreData="BHSA",
        coreVersion=VERSION,
    ),
)

And now we produce the TF.

In [59]:
TF = Fabric(locations=TFDIR, silent=True)
TF.save(nodeFeatures=nodeFeatures, metaData=metaData)

True

# Done
Now you can write notebooks to process BHSA data and grab the OSM morphology as you go, like so:

```
BHSA = 'BHSA/tf/2021'
OSM = 'bridging/tf/2021'

TF = Fabric(locations='~/github/etcbc', modules=[BHSA, OSM])
api = TF.load('''
    sp
    osm
''')
api.makeAvailableIn(globals())
```